## Data Source and Structure

The objective of the carbon footprint model is to estimate the carbon emissions associated with daily travel for each household. The process involves:
<ul>
    <li>Merging the VISTA tables to associate trips and stops with household and personal attributes</li>
    <li>Assigning CO2 emissions per kilometer to each trip based on travel mode, using predefined emissions factors</li>
    <li>Calculating total household emissions by aggregating emissions from individual trips</li>
    <li>Building a linear regression model to identify which household characteristics significantly affect total emissions</li>
</ul>

## Carbon Footprint Model for Household Travel <a name="second-bullet"></a>

### Data Preparation

##### 1. Trip Data Flattening

<p style='text-align: justify'>
Each trip in the dataset can include up to nine mode/distance pairs. To make the data easier to analyze, each trip is split into individual segments (e.g., walking → bus → walking becomes three segments). Only valid numeric distances are kept to ensure data quality. This process of flattening the data—converting columns into separate rows—is done using SQL. The code is shown below.

```sql
-- Step 1: Explode trip records into individual segments for each mode/distance pair
drop table if exists exploded_segments;
-- Repeat for each of the 9 possible segments (mode1 to mode9)
-- Only include segments where distance is a valid number
create temp table exploded_segments as
-- For mode1 and dist1
select t.tripid, p.persid, h.hhid, 1 as segment_no, t.mode1 as mode,
       case when t.dist1 ~ '^[0-9.]+$' then t.dist1::float else null end as dist_segment
from t join p on t.persid = p.persid join h on p.hhid = h.hhid
where t.mode1 is not null and t.dist1 ~ '^[0-9.]+$'
-- Repeat block for mode2 to mode9
-- For mode2 and dist2
union all
select t.tripid, p.persid, h.hhid, 2, t.mode2,
       case when t.dist2 ~ '^[0-9.]+$' then t.dist2::float else null end
from t join p on t.persid = p.persid join h on p.hhid = h.hhid
where t.mode2 is not null and t.dist2 ~ '^[0-9.]+$'
--For mode3 and dist3
union all
select t.tripid, p.persid, h.hhid, 3, t.mode3,
       case when t.dist3 ~ '^[0-9.]+$' then t.dist3::float else null end
from t join p on t.persid = p.persid join h on p.hhid = h.hhid
where t.mode3 is not null and t.dist3 ~ '^[0-9.]+$'
--For mode4 and dist4
union all
select t.tripid, p.persid, h.hhid, 4, t.mode4,
       case when t.dist4 ~ '^[0-9.]+$' then t.dist4::float else null end
from t join p on t.persid = p.persid join h on p.hhid = h.hhid
where t.mode4 is not null and t.dist4 ~ '^[0-9.]+$'
--For mode5 and dist5
union all
select t.tripid, p.persid, h.hhid, 5, t.mode5,
       case when t.dist5 ~ '^[0-9.]+$' then t.dist5::float else null end
from t join p on t.persid = p.persid join h on p.hhid = h.hhid
where t.mode5 is not null and t.dist5 ~ '^[0-9.]+$'
--For mode6 and dist6
union all
select t.tripid, p.persid, h.hhid, 6, t.mode6,
       case when t.dist6 ~ '^[0-9.]+$' then t.dist6::float else null end
from t join p on t.persid = p.persid join h on p.hhid = h.hhid
where t.mode6 is not null and t.dist6 ~ '^[0-9.]+$'
--For mode7 and dist7
union all
select t.tripid, p.persid, h.hhid, 7, t.mode7,
       case when t.dist7 ~ '^[0-9.]+$' then t.dist7::float else null end
from t join p on t.persid = p.persid join h on p.hhid = h.hhid
where t.mode7 is not null and t.dist7 ~ '^[0-9.]+$'
--For mode8 and dist8
union all
select t.tripid, p.persid, h.hhid, 8, t.mode8,
       case when t.dist8 ~ '^[0-9.]+$' then t.dist8::float else null end
from t join p on t.persid = p.persid join h on p.hhid = h.hhid
where t.mode8 is not null and t.dist8 ~ '^[0-9.]+$'
--For mode9 and dist9
union all
select t.tripid, p.persid, h.hhid, 9, t.mode9,
       case when t.dist9 ~ '^[0-9.]+$' then t.dist9::float else null end
from t join p on t.persid = p.persid join h on p.hhid = h.hhid
where t.mode9 is not null and t.dist9 ~ '^[0-9.]+$';
```

##### 2. Mapping Transport Modes to Emission Categories

<p style='text-align: justify'>
Travel modes from the VISTA data (e.g., “Vehicle Driver”, “Public Bus”) are mapped into a simplified set of categories such as “Average Victorian Car” or “Bus” that match available emission factors in grams per kilometer.

```sql
-- Step 2: Map transport modes from VISTA data to emission factor categories
drop table if exists mapped_segments;

create temp table mapped_segments as
select *,
  case
    when mode = 'Vehicle Driver' then 'Average Victorian Car'
    when mode = 'Vehicle Passenger' then 'Dual Occupancy Car'
    when mode IN ('Public Bus', 'School Bus', 'Bus') then 'Bus'
    when mode = 'Train' then 'Train'
    when mode = 'Tram' then 'Tram'
    when mode = 'Walking' then 'Walking'
    when mode = 'Bicycle' then 'Bike'
    when mode = 'Motorcycle' then 'Motorcycle'
    when mode = 'Taxi' then 'Average Victorian Car'
    when mode = 'Mobility Scooter' then 'Walking'
    when mode = 'Other' then 'Average Victorian Car'
    else NULL
  end as mapped_mode
from exploded_segments
where dist_segment > 0; --Exclude zero or null distances

##### 3. Calculating Segment Level Emissions

For each segment, emissions are computed using the formula:

\begin{align}
CO_2 \; Emissions \; (kg)  =  \; \frac {Distance \; (km)  \; ×  \; Emission Factor \; (g/km)}{1000}
\end{align}
 
These emissions are stored per trip segment along with household IDs for aggregation.

```sql
-- Step 3: Calculate segment level emissions
-- Segment is a portion of a trip using one mode of transport with an associated distance
-- For example, if a person walks to the bus stop (segment 1), takes a bus (segment 2), 
-- and then walks again (segment 3), that trip has 3 segments. 
drop table if exists segment_emissions;

create temp table segment_emissions as
select 
  m.hhid,
  m.tripid,
  m.mode,
  m.mapped_mode,
  m.dist_segment,
  (m.dist_segment * e.co2_g_per_km / 1000.0) as co2_emissions_kg --Convert gram/km to kilogram
from mapped_segments m
left join emission_factors e on m.mapped_mode = e.mode
where m.mapped_mode is not null;

##### 4. Aggregating Household Emissions 

<p style='text-align: justify'>
Emissions and total distance are aggregated at the household level, allowing insights into which households generate higher transport emissions. This table also includes metadata like household size, income, vehicle ownership, and dwelling type.

```sql
-- Step 4: Aggregate emissions and distances by household
drop table if exists household_emissions;

create temp table household_emissions as
select 
  hhid,
  sum(co2_emissions_kg) as total_household_emissions_kg,
  count(*) as total_segments,
  sum(dist_segment) as total_distance_km
from segment_emissions
group by hhid
order by total_household_emissions_kg desc;

##### 5. Validation and Quality Checks

<ul>
<li><b>Validation 1</b>: Compare Trip-Level Segment Distance to Reported Cumulative Distance</li>
<li><b>Validation 2</b>: Compare Trip Counts from Raw and Processed Data</li>
<li><b>Validation 3</b>: Direct Check of Segment Counts and Cumulative Distance in Raw Trip Data</li>
</ul>

```sql
-- Validation 1: Compare the sum of all segment distances to the reported cumulative trip distance
-- Helps detect data inconsistencies or missing segments

-- First, calculate the sum of all valid segment distances for each trip
with segment_distance_per_trip as (
  select tripid, sum(dist_segment) as sum_segment_dist
  from segment_emissions
  group by tripid
),

-- Then extract the reported cumulative trip distance
cumdist_per_trip as (
  select tripid, cumdist::float as cumdist
  from t
  where cumdist ~ '^[0-9.]+$'
)

-- Compare the two values and compute the difference
select 
  v.tripid,
  v.cumdist,
  s.sum_segment_dist,
  ROUND((v.cumdist - s.sum_segment_dist)::numeric, 2) as difference_km
from cumdist_per_trip v
left join segment_distance_per_trip s on v.tripid = s.tripid
order by abs(v.cumdist - s.sum_segment_dist) desc
limit 50;

```sql
-- Validation 2: Check if number of trips per household in the original table
-- matches the number of trips with valid segment data
with segment_counts as (
  select hhid, count(distinct tripid) as trips_with_segments, count(*) as total_segments
  from segment_emissions
  group by hhid
),

-- Total trips per household from original trip table
trip_counts as (
  select h.hhid, count(distinct t.tripid) as total_trips
  from h
  left join p on h.hhid = p.hhid
  left join t on p.persid = t.persid
  group by h.hhid
)

-- Compare them
select 
  t.hhid,
  t.total_trips,
  s.trips_with_segments,
  s.total_segments
from trip_counts t
left join segment_counts s on t.hhid = s.hhid
order by t.total_trips desc;

```sql
-- Validation 3: Check segment count and segment distance directly from the trip table
-- Calculates total valid segments and compares to the reported cumulative distance (cumdist)
select 
  t.tripid,
  h.hhid,

  -- Count valid numeric segment distances
  (case when t.dist1 ~ '^[0-9.]+$' then 1 else 0 end +
   case when t.dist2 ~ '^[0-9.]+$' then 1 else 0 end +
   case when t.dist3 ~ '^[0-9.]+$' then 1 else 0 end +
   case when t.dist4 ~ '^[0-9.]+$' then 1 else 0 end +
   case when t.dist5 ~ '^[0-9.]+$' then 1 else 0 end +
   case when t.dist6 ~ '^[0-9.]+$' then 1 else 0 end +
   case when t.dist7 ~ '^[0-9.]+$' then 1 else 0 end +
   case when t.dist8 ~ '^[0-9.]+$' then 1 else 0 end +
   case when t.dist9 ~ '^[0-9.]+$' then 1 else 0 end) as segment_count,

  -- Sum of segment distances (only valid numerics)
  (
    case when t.dist1 ~ '^[0-9.]+$' then t.dist1::float else 0 end +
    case when t.dist2 ~ '^[0-9.]+$' then t.dist2::float else 0 end +
    case when t.dist3 ~ '^[0-9.]+$' then t.dist3::float else 0 end +
    case when t.dist4 ~ '^[0-9.]+$' then t.dist4::float else 0 end +
    case when t.dist5 ~ '^[0-9.]+$' then t.dist5::float else 0 end +
    case when t.dist6 ~ '^[0-9.]+$' then t.dist6::float else 0 end +
    case when t.dist7 ~ '^[0-9.]+$' then t.dist7::float else 0 end +
    case when t.dist8 ~ '^[0-9.]+$' then t.dist8::float else 0 end +
    case when t.dist9 ~ '^[0-9.]+$' then t.dist9::float else 0 end
  ) as segment_sum_km,

  -- Trip-level cumulative distance
  case when t.cumdist ~ '^[0-9.]+$' then t.cumdist::float else null end as cumdist_km,

  -- Difference between cumulative and segment sum
  ROUND((
    case when t.cumdist ~ '^[0-9.]+$' then t.cumdist::float else 0 end - (
      case when t.dist1 ~ '^[0-9.]+$' then t.dist1::float else 0 end +
      case when t.dist2 ~ '^[0-9.]+$' then t.dist2::float else 0 end +
      case when t.dist3 ~ '^[0-9.]+$' then t.dist3::float else 0 end +
      case when t.dist4 ~ '^[0-9.]+$' then t.dist4::float else 0 end +
      case when t.dist5 ~ '^[0-9.]+$' then t.dist5::float else 0 end +
      case when t.dist6 ~ '^[0-9.]+$' then t.dist6::float else 0 end +
      case when t.dist7 ~ '^[0-9.]+$' then t.dist7::float else 0 end +
      case when t.dist8 ~ '^[0-9.]+$' then t.dist8::float else 0 end +
      case when t.dist9 ~ '^[0-9.]+$' then t.dist9::float else 0 end
    )
  )::numeric, 2) as difference_km

from h
join p on h.hhid = p.hhid
join t on p.persid = t.persid

order by abs(
  (case when t.cumdist ~ '^[0-9.]+$' then t.cumdist::float else 0 end) - (
    case when t.dist1 ~ '^[0-9.]+$' then t.dist1::float else 0 end +
    case when t.dist2 ~ '^[0-9.]+$' then t.dist2::float else 0 end +
    case when t.dist3 ~ '^[0-9.]+$' then t.dist3::float else 0 end +
    case when t.dist4 ~ '^[0-9.]+$' then t.dist4::float else 0 end +
    case when t.dist5 ~ '^[0-9.]+$' then t.dist5::float else 0 end +
    case when t.dist6 ~ '^[0-9.]+$' then t.dist6::float else 0 end +
    case when t.dist7 ~ '^[0-9.]+$' then t.dist7::float else 0 end +
    case when t.dist8 ~ '^[0-9.]+$' then t.dist8::float else 0 end +
    case when t.dist9 ~ '^[0-9.]+$' then t.dist9::float else 0 end
  )
) desc;

##### 6. Merging with Household Attributes

<p style='text-align: justify'>
Segment emissions are enriched with household demographic attributes to allow further socioeconomic and behavioural analysis. Final metrics per household include:
<ul>
<li>Total emissions (kg CO₂)</li>
<li>Total travel distance (km)</li>
<li>Number of valid trip segments</li>
</ul>

```sql
-- Calculate CO2 emissions for each trip segments without household features
select
  m.tripid,
  m.persid,
  m.hhid,
  m.segment_no,
  m.mode,
  m.mapped_mode,
  m.dist_segment,
  e.co2_g_per_km,
  ROUND((m.dist_segment * e.co2_g_per_km / 1000.0)::numeric, 4) as co2_emissions_kg
from mapped_segments m
left join emission_factors e on m.mapped_mode = e.mode
where m.mapped_mode is not null
order by m.tripid, m.segment_no;

-- Combine segment data with houshold features
select
  h.hhid,
  h.hhsize,
  h.hhinc,
  h.dwelltype,
  h.owndwell,
  h.yearslived,
  h.monthslived,
  h.totalbikes,
  h.totalvehs,
  h.youngest,
  h.aveage,
  h.oldest,
  h.homesa2,
  h.homepc,
  m.tripid,
  m.persid,
  m.segment_no,
  m.mode,
  m.mapped_mode,
  m.dist_segment,
  e.co2_g_per_km,
  ROUND((m.dist_segment * e.co2_g_per_km / 1000.0)::numeric, 4) as co2_emissions_kg

from mapped_segments m
left join emission_factors e on m.mapped_mode = e.mode
join h on m.hhid = h.hhid
order by h.hhid, m.tripid, m.segment_no;

-- Create temporary table of emissions linked with household features
-- The purpose of this table is to combine segment_emissions data with household characteristics
-- for detailed per-segment analysis
drop table if exists household_segment_emissions;

create temp table household_segment_emissions as
select 
  h.hhid,
  h.hhsize,
  h.hhinc,
  h.dwelltype,
  h.owndwell,
  h.yearslived,
  h.monthslived,
  h.totalbikes,
  h.cars,
  h.fourwds,
  h.utes,
  h.vans,
  h.trucks,
  h.mbikes,
  h.othervehs,
  h.totalvehs,
  h.youngest,
  h.aveage,
  h.oldest,
  h.homesa2,
  h.homepc,
  se.co2_emissions_kg, -- segment level emissions
  se.dist_segment -- segment level distance
from h
join (
  select 
    hhid,
    tripid,
    dist_segment,
    co2_emissions_kg
  from segment_emissions
) se on h.hhid = se.hhid;

-- Aggregate emissions data by household id (hhid), preserving all households
-- This will produce final results per household with total CO2 emissions in kg,
-- total distance traveled in km, total number of trip segments, along with household attributes
-- Add total number of trips per household to emissions + household data
select 
  h.hhid,
  h.sampleregion,
  h.hhsize,
  h.hhinc,
  h.dwelltype,
  h.owndwell,
  h.yearslived,
  h.monthslived,
  h.totalbikes,
  h.cars,
  h.fourwds,
  h.utes,
  h.vans,
  h.trucks,
  h.mbikes,
  h.othervehs,
  h.totalvehs,
  h.youngest,
  h.aveage,
  h.oldest,
  h.homesa2,
  h.homepc,
  coalesce(sum(se.co2_emissions_kg), 0) as total_emissions_kg,     -- total kg of CO2
  coalesce(sum(se.dist_segment), 0) as total_distance_km,          -- total trip distance
  count(se.tripid) as total_segments,                              -- count of matched segments
  coalesce(trip_counts.total_trips, 0) as total_trips              -- new: total number of trips from 't'
from h
left join segment_emissions se on h.hhid = se.hhid
left join (
    select hhid, count(*) as total_trips
    from t
    group by hhid
) as trip_counts on h.hhid = trip_counts.hhid
group by 
  h.hhid, h.sampleregion, h.hhsize, h.hhinc, h.dwelltype, h.owndwell, h.yearslived,
  h.monthslived, h.totalbikes, h.cars, h.fourwds, h.utes, h.vans, h.trucks,
  h.mbikes, h.othervehs, h.totalvehs, h.youngest, h.aveage, h.oldest,
  h.homesa2, h.homepc, trip_counts.total_trips
order by h.hhid;
-- This result is saved as "household_emissions.csv" 

### Carbon Footprint Model for Household Travel

<p style='text-align: justify'>
Carbon footprint model developed using linear regression model with the target value is CO2 emissions for each household from direct calculation done in SQL. The model have 10 explanatory variables from household characteristics. The goal is to measrue the effect of each variables to the emissions produced by the households. The steps is as follows.

In [ ]:
import pandas as pd
import numpy as np
from pandas.api.types import CategoricalDtype
import statsmodels.api as sm

In [ ]:
#load the data
df = pd.read_csv('household_emissions.csv')

In [ ]:
#View the data
df.head()

,hhid,sampleregion,hhsize,hhinc,dwelltype,owndwell,yearslived,monthslived,totalbikes,cars,...,totalvehs,youngest,aveage,oldest,homesa2,homepc,total_emissions_kg,total_distance_km,total_segments,total_trips
0,Y12H0000101,North East Melbourne,4,1225,Separate House,Fully Owned,7.0,9,2,2,...,2,6,27,50,Mill Park - North,3082,9.642290,47.390,11,11
1,Y12H0000102,North East Melbourne,4,1700,Separate House,Being Purchased,9.0,4,0,3,...,3,21,40,57,Mill Park - North,3082,26.543365,149.898,14,10
2,Y12H0000103,North East Melbourne,4,1000,Separate House,Being Purchased,8.0,5,0,2,...,2,13,30,47,Mill Park - North,3082,15.251175,80.582,14,10
3,Y12H0000104,North East Melbourne,3,1875,Separate House,Fully Owned,3.0,0,2,3,...,3,22,43,56,Mill Park - North,3082,28.107702,115.290,9,9
4,Y12H0000107,North East Melbourne,3,1750,Separate House,Being Purchased,8.0,2,3,1,...,1,9,21,41,Mill Park - North,3082,2.535520,34.980,12,12


In [ ]:
#View the data type
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18152 entries, 0 to 18151
Data columns (total 26 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   hhid                18152 non-null  object 
 1   sampleregion        18152 non-null  object 
 2   hhsize              18152 non-null  int64  
 3   hhinc               18152 non-null  object 
 4   dwelltype           18152 non-null  object 
 5   owndwell            18152 non-null  object 
 6   yearslived          18152 non-null  float64
 7   monthslived         18152 non-null  int64  
 8   totalbikes          18152 non-null  int64  
 9   cars                18152 non-null  int64  
 10  fourwds             18152 non-null  int64  
 11  utes                18152 non-null  int64  
 12  vans                18152 non-null  int64  
 13  trucks              18152 non-null  int64  
 14  mbikes              18152 non-null  int64  
 15  othervehs           18152 non-null  int64  
 16  tota

### Two Model Comparison

<p style='text-align: justify'>
Two carbon footprint models will be created. The first model will use only numerical variables, while the second will include both categorical and numerical variables. The adjusted R² will be assessed for both models.

In [ ]:
# === 1. Define variables for both models ===
numerical_vars = ["hhsize", "hhinc", "yearslived", "totalbikes", "totalvehs", "youngest", "aveage", "total_trips"]
categorical_vars = ["dwelltype", "owndwell", "sampleregion"]
target_var = "total_emissions_kg"

# Combine all variables
all_vars = numerical_vars + categorical_vars + [target_var]

# === 2. Drop rows with missing values in any relevant column ===
df_shared = df[all_vars].dropna().copy()

# === 3. Prepare Model 1: Numerical variables only ===
x1 = df_shared[numerical_vars]
x1 = sm.add_constant(x1)
y1 = df_shared[target_var]

# Ensure all columns in x1 and y1 are numeric
x1 = x1.apply(pd.to_numeric, errors='coerce')
y1 = pd.to_numeric(y1, errors='coerce')

# Drop rows with any missing values
valid_idx = x1.dropna().index.intersection(y1.dropna().index)

print("x1 shape:", x1.shape)
print("y1 shape:", y1.shape)
print("x1 index head:", x1.index[:5])
print("y1 index head:", y1.index[:5])

x1 = x1.loc[valid_idx]
y1 = y1.loc[valid_idx]

model1 = sm.OLS(y1, x1).fit()
print(model1.summary())

# === 4. Prepare Model 2: Numerical + One-Hot Encoded Categorical ===

# Reset reference categories
df_shared["dwelltype"] = df_shared["dwelltype"].astype(CategoricalDtype(
    categories=["Flat or Apartment", "Other", "Separate House", "Terrace/Townhouse"],
    ordered=False
))
df_shared["owndwell"] = df_shared["owndwell"].astype(CategoricalDtype(
    categories=["Fully Owned", "Something Else", "Being Purchased", "Occupied Rent-Free", "Being Rented"],
    ordered=False
))
df_shared["sampleregion"] = df_shared["sampleregion"].astype(CategoricalDtype(
    categories=[
        "East Melbourne", "Geelong", "North West Melbourne",
        "West Melbourne", "Inner Melbourne", "North East Melbourne"
    ],
    ordered=False
))

# One-hot encode categorical variables
df_encoded = pd.get_dummies(df_shared, columns=categorical_vars, drop_first=True)

# Ensure ALL columns are numeric (object → NaN)
df_encoded = df_encoded.apply(pd.to_numeric, errors='coerce')

# === Clean x2 and y2 ===

# Drop rows with any missing values first
df_encoded_clean = df_encoded.dropna()

# Separate predictors and target
y2 = df_encoded_clean[target_var]
x2 = df_encoded_clean.drop(columns=[target_var])

# Convert all to numeric (again, just to be safe)
x2 = x2.apply(pd.to_numeric, errors='coerce')
y2 = pd.to_numeric(y2, errors='coerce')

# Ensure all values are numeric and drop problematic rows
valid_index = x2.select_dtypes(include=[np.number]).dropna().index.intersection(y2.dropna().index)
x2 = x2.loc[valid_index]
y2 = y2.loc[valid_index]

# Double-check all columns are numeric now
assert all(x2.dtypes != 'object'), "Non-numeric column found in x2"
assert y2.dtypes != 'object', "Non-numeric column found in y2"

print(x2.dtypes[x2.dtypes == 'object'])

print("x2 shape:", x2.shape)
print("y2 shape:", y2.shape)
print("x2 index head:", x2.index[:5])
print("y2 index head:", y2.index[:5])

# Add constant and force float conversion
x2 = sm.add_constant(x2).astype(float)
y2 = y2.astype(float)

# Fit model
model2 = sm.OLS(y2, x2).fit()
print(model2.summary())

x1 shape: (18152, 9)
y1 shape: (18152,)
x1 index head: RangeIndex(start=0, stop=5, step=1)
y1 index head: RangeIndex(start=0, stop=5, step=1)
                            OLS Regression Results                            
Dep. Variable:     total_emissions_kg   R-squared:                       0.229
Model:                            OLS   Adj. R-squared:                  0.229
Method:                 Least Squares   F-statistic:                     619.2
Date:                Wed, 30 Apr 2025   Prob (F-statistic):               0.00
Time:                        06:15:03   Log-Likelihood:                -70121.
No. Observations:               16665   AIC:                         1.403e+05
Df Residuals:                   16656   BIC:                         1.403e+05
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025 

In [ ]:
# Predict trips using the second model (with constant)
# Add predictions to the shared dataset
df_model1_results = x1.copy()
df_model1_results["actual_emissions"] = y1
df_model1_results["pred_model1"] = model1.predict(x1)

df_model2_results = x2.copy()
df_model2_results["actual_emissions"] = y2
df_model2_results["pred_model2"] = model2.predict(x2)

# Join both predictions on index (only valid if both models used same rows)
comparison_df = pd.DataFrame({
    "actual_emissions": y1,
    "pred_model1": model1.predict(x1),
    "pred_model2": model2.predict(x2)
})

# Show first 20 rows
print("\nComparison of Observed vs Predicted (Model 1 vs Model 2):")
print(comparison_df.head(20))


Comparison of Observed vs Predicted (Model 1 vs Model 2):
    actual_emissions  pred_model1  pred_model2
0           9.642290    16.335949    17.360195
1          26.543365    19.922061    22.577416
2          15.251175    15.152803    18.175207
3          28.107702    20.069580    20.871085
4           2.535520    14.462758    17.722305
5          30.353441    19.579334    22.461410
6           9.364358    16.069943    18.718961
7          14.694312    13.797224    16.988804
8          15.761670    15.213956    18.654102
9          31.647678    16.557402    19.367947
10         10.144518    20.575887    22.551383
11          3.883734     7.518159     9.455511
12         18.480040    18.723554    21.338382
13          2.164944     9.770323    12.767612
14         10.439516    12.712409    14.280528
15         19.016400    13.574530    15.309898
16          6.114504    14.399678    15.691859
18          3.252292     7.869712     9.393418
19         20.536493    15.010137    16.572309
2

### 1.4 Discussion

<p style='text-align: justify'>
The regression analysis compared two linear models to predict household carbon emissions using the VISTA dataset. <b>Model 1</b> used only <b>numerical variables</b> such as household size, income, years lived, number of vehicles, and average age, while <b>Model 2</b> incorporated both <b>numerical and categorical variables</b>, including dwelling type, ownership status, and sample region via one-hot encoding. <b>Model 1 achieved an R-squared of 0.150</b>, indicating that 15% of the variation in household emissions could be explained by the numerical predictors. <b>The most significant variables were household size, income, years lived, number of bikes and vehicles, while variables like youngest and average age were not statistically significant.</b>
<p style='text-align: justify'>
<b>Model 2 provided a slightly better fit, with an R-squared of 0.160</b>, suggesting that incorporating categorical factors explained an additional 1% of the variance in emissions. Notably, households living in separate houses, those in the process of purchasing their dwelling, and those located in Geelong or North East Melbourne were associated with higher emissions. Meanwhile, residents in West and Inner Melbourne showed significantly lower emissions. <b>Despite the improved R-squared, Model 2 came with increased complexity</b>, as it involved 19 predictors compared to 8 in Model 1, and had fewer valid observations due to missing values in categorical variables.
<p style='text-align: justify'>
A comparison of predicted emissions for both models revealed that Model 2 generally provided predictions closer to actual emissions, especially for higher-emitting households. This indicates that the added categorical context contributed to a more nuanced understanding of household travel behavior. However, the marginal gain in explanatory power must be weighed against the trade-off in model simplicity and interpretability. Overall, Model 2 may be preferred for richer insights, while Model 1 offers a more streamlined and interpretable alternative.

## References

- Nassir, N. (2025). *Lecture Week 1* [Lecture slides]. University of Melbourne, Canvas.
- Nassir, N. (2025). *Lecture Week 2: VISTA and SQL* [Lecture slides]. University of Melbourne, Canvas.
- Nassir, N. (2025). *Week 3: Python basics* [Lecture slides]. University of Melbourne, Canvas.
- Nassir, N. (2025). *Lecture Week 4: Trip Generation Models* [Lecture slides]. University of Melbourne, Canvas.
- Ortúzar, J. de D., & Willumsen, L. G. (2011). *Modelling Transport* (5th ed.). Wiley.
- University of Melbourne. (2025). *CVEN90063: Transport system modelling - Subject project 1* [Lecture slides]. Canvas.